<a href="https://colab.research.google.com/github/vtRodrigues92/Analise_de_dados_com_python/blob/master/EXEMPLO_COLETA_DADOS_SPARK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##AULA 4 - MANIPULAÇÃO DE DADOS_2

###INSTALANDO BIBLIOTECAS

In [ ]:
pip install azure-storage-blob

     |████████████████████████████████| 346 kB 16.2 MB/s 
     |████████████████████████████████| 179 kB 60.1 MB/s 
     |████████████████████████████████| 3.6 MB 45.8 MB/s 
     |████████████████████████████████| 85 kB 4.0 MB/s 
     |████████████████████████████████| 41 kB 615 kB/s 
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 3.10.0.2
    Uninstalling typing-extensions-3.10.0.2:
      Successfully uninstalled typing-extensions-3.10.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.0 requires tf-estimator-nightly==2.8.0.dev2021122109, which is not installed.
arviz 0.11.4 requires typing-extensions<4,>=3.7.4.3, but you have typing-extensions 4.1.1 which is incompatible.


In [ ]:
pip install pyspark

     |████████████████████████████████| 281.4 MB 33 kB/s 
     |████████████████████████████████| 198 kB 32.1 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=ba3d2a577bbd6f9590e248ec825b408061cadde4c6849c79d47e3aa60f6447a2
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


###IMPORTANDO AS LIBS PARA USO

In [ ]:
import requests
from pyspark.sql import SparkSession
from pyspark.sql.types import *

###CRIANDO UMA APLICAÇÃO SPARK

In [ ]:
spark = SparkSession.builder.getOrCreate()

###FUNÇÃO DOWNLOAD DE DADOS

In [ ]:
def download(url_arquivo, nome_arquivo):
  requisicao = requests.get(url_arquivo)
  conteudo = requisicao.content
  arquivo = open(nome_arquivo,'wb')
  arquivo.write(conteudo)
  arquivo.close()

###FAZENDO DOWNLOADS DOS ARQUIVOS DE ORDENS DE COMPRA E CLIENTES

In [ ]:
url_clientes = "https://datalakeaula141600.blob.core.windows.net/datalake/raw-zone/olist/olist_customers_dataset/olist_customers_dataset.csv"
url_ordem_compra = "https://datalakeaula141600.blob.core.windows.net/datalake/raw-zone/olist/olist_orders_dataset/olist_orders_dataset.csv"
nome_arquivo_clientes = "olist_customers_dataset.csv"
nome_arquivo_ordem_compra = "olist_orders_dataset.csv"
download(url_clientes, nome_arquivo_clientes)
download(url_ordem_compra, nome_arquivo_ordem_compra)

##LENDO A TABELA ORDERS

###ALTERANDO O TIPO DE DADOS DAS COLUNAS

In [ ]:
schema_orders = StructType([
  
  StructField("order_id", StringType(), True),
  StructField("customer_id", StringType(), True),
  StructField("order_status", StringType(), True),
  StructField("order_purchase_timestamp", TimestampType(), True),
  StructField("order_approved_at", TimestampType(), True),
  StructField("order_delivered_carrier_date", TimestampType(), True),
  StructField("order_delivered_customer_date", TimestampType(), True),
  StructField("order_estimated_delivery_date", TimestampType(), True),                      

])

###DEFININDO A VARIAVEL COM O NOVO TIPO DE DADOS 

In [ ]:
df_orders = spark.read.csv(nome_arquivo_ordem_compra, header=True, schema=schema_orders)

###MOSTRANDO AS 5 PRIMEIRAS LINHAS DO DATA_FRAME

In [ ]:
df_orders.show(truncate=False, n=5)

+--------------------------------+--------------------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|order_id                        |customer_id                     |order_status|order_purchase_timestamp|order_approved_at  |order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|
+--------------------------------+--------------------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|e481f51cbdc54678b7cc49136f2d6af7|9ef432eb6251297304e76186b10a928d|delivered   |2017-10-02 10:56:33     |2017-10-02 11:07:15|2017-10-04 19:55:00         |2017-10-10 21:25:13          |2017-10-18 00:00:00          |
|53cdb2fc8bc7dce0b6741e2150273451|b0830fb4747a6c6d20dea0b8c802d7ef|delivered   |2018-07-24 20:41:37     |2018-07-26 03:24:27|2018-07-26 14:3

###MOSTRANDO O DATA_FRAME NO MODO VERTICAL

In [ ]:
df_orders.show(truncate=False, vertical=True, n=2)

-RECORD 0---------------------------------------------------------
 order_id                      | e481f51cbdc54678b7cc49136f2d6af7 
 customer_id                   | 9ef432eb6251297304e76186b10a928d 
 order_status                  | delivered                        
 order_purchase_timestamp      | 2017-10-02 10:56:33              
 order_approved_at             | 2017-10-02 11:07:15              
 order_delivered_carrier_date  | 2017-10-04 19:55:00              
 order_delivered_customer_date | 2017-10-10 21:25:13              
 order_estimated_delivery_date | 2017-10-18 00:00:00              
-RECORD 1---------------------------------------------------------
 order_id                      | 53cdb2fc8bc7dce0b6741e2150273451 
 customer_id                   | b0830fb4747a6c6d20dea0b8c802d7ef 
 order_status                  | delivered                        
 order_purchase_timestamp      | 2018-07-24 20:41:37              
 order_approved_at             | 2018-07-26 03:24:27          

###EXIBINDO OS TIPOS DE DADOS EM FORMATO DE ARVORE

In [ ]:
df_orders.printSchema()

root
 |-- order_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- order_status: string (nullable = true)
 |-- order_purchase_timestamp: timestamp (nullable = true)
 |-- order_approved_at: timestamp (nullable = true)
 |-- order_delivered_carrier_date: timestamp (nullable = true)
 |-- order_delivered_customer_date: timestamp (nullable = true)
 |-- order_estimated_delivery_date: timestamp (nullable = true)



###EXIBINDO A MAIOR DATA DE UMA COLUNA DO DATA_FRAME

In [ ]:
df_orders.agg({"order_purchase_timestamp" : "max"}).collect()

[Row(max(order_purchase_timestamp)=datetime.datetime(2018, 10, 17, 17, 30, 18))]

###CRIANDO UMA TEMPVIEW DO DATA_FRAME

In [ ]:
df_orders.createOrReplaceTempView("orders")

###MOSTRANDO A VIEW CRIADA COM COMANDO SQL (SPARK)

In [ ]:
spark.sql("""
SELECT *
FROM   orders""").show()

+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|            order_id|         customer_id|order_status|order_purchase_timestamp|  order_approved_at|order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|
+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|e481f51cbdc54678b...|9ef432eb625129730...|   delivered|     2017-10-02 10:56:33|2017-10-02 11:07:15|         2017-10-04 19:55:00|          2017-10-10 21:25:13|          2017-10-18 00:00:00|
|53cdb2fc8bc7dce0b...|b0830fb4747a6c6d2...|   delivered|     2018-07-24 20:41:37|2018-07-26 03:24:27|         2018-07-26 14:31:00|          2018-08-07 15:27:45|          2018-08-13 00:00:00|
|47770eb9100c2d0c4...|41ce2a54c0b03bf34...|  

###EXIBINDO A MAIOR DATA DE UMA COLUNA USANDO SQL (SPARK)

In [ ]:
spark.sql("""
SELECT MAX(order_purchase_timestamp)
FROM   orders""").show()

+-----------------------------+
|max(order_purchase_timestamp)|
+-----------------------------+
|          2018-10-17 17:30:18|
+-----------------------------+



##LENDO A TABELA CLIENTES

In [ ]:
df_clientes = spark.read.csv(nome_arquivo_clientes, header = True, inferSchema = True)

###MOSTRANDO O TIPO DE DADOS DAS COLUNAS NO EFEITO ARVORE

In [ ]:
df_clientes.printSchema()

root
 |-- customer_id: string (nullable = true)
 |-- customer_unique_id: string (nullable = true)
 |-- customer_zip_code_prefix: integer (nullable = true)
 |-- customer_city: string (nullable = true)
 |-- customer_state: string (nullable = true)



###EXIBINDO A TABELA CLIENTES COM CABEÇALHO

In [ ]:
df_clientes.show(truncate=False)

+--------------------------------+--------------------------------+------------------------+---------------------+--------------+
|customer_id                     |customer_unique_id              |customer_zip_code_prefix|customer_city        |customer_state|
+--------------------------------+--------------------------------+------------------------+---------------------+--------------+
|06b8999e2fba1a1fbc88172c00ba8bc7|861eff4711a542e4b93843c6dd7febb0|14409                   |franca               |SP            |
|18955e83d337fd6b2def6b18a428ac77|290c77bc529b7ac935b93aa66c333dc3|9790                    |sao bernardo do campo|SP            |
|4e7b3e00288586ebd08712fdd0374a03|060e732b5b29e8181a18229c7b0b2b5e|1151                    |sao paulo            |SP            |
|b2b6027bc5c5109e529d4dc6358b12c3|259dac757896d24d7702b9acbbff3f3c|8775                    |mogi das cruzes      |SP            |
|4f2d8ab171c80ec8364f7c12e35b23ad|345ecd01c38d18a9036ed96c73b8d066|13056                  

###CRIANDO UMA VIEW TEMPORÁRIA DA TABELA CLIENTES

In [ ]:
df_clientes.createOrReplaceTempView("clientes")

###CRIANDO UMA COLUNA REGIÇÃO, DEFININDO A REGIÃO GEOGRÁFICA A PARTIR DOS ESTADOS

In [ ]:
spark.sql("""
SELECT  *,
        CASE
          WHEN customer_state IN ('MG', 'SP', 'RJ', 'ES') THEN 'Sudeste' 
          WHEN customer_state IN ('SC', 'RS', 'PR') THEN 'Sul' 
          WHEN customer_state IN ('DF', 'GO', 'MT', 'MS') THEN 'Centro-Oeste'
          WHEN customer_state IN ('AC', 'AM', 'AP', 'RO', 'RR', 'PA', 'TO') THEN 'Norte'
          ELSE 'Nordeste'
        END AS customer_region
FROM    clientes
"""

).show()

+--------------------+--------------------+------------------------+--------------------+--------------+---------------+---------------+
|         customer_id|  customer_unique_id|customer_zip_code_prefix|       customer_city|customer_state|customer_region|customer_region|
+--------------------+--------------------+------------------------+--------------------+--------------+---------------+---------------+
|06b8999e2fba1a1fb...|861eff4711a542e4b...|                   14409|              franca|            SP|        Sudeste|        Sudeste|
|18955e83d337fd6b2...|290c77bc529b7ac93...|                    9790|sao bernardo do c...|            SP|        Sudeste|        Sudeste|
|4e7b3e00288586ebd...|060e732b5b29e8181...|                    1151|           sao paulo|            SP|        Sudeste|        Sudeste|
|b2b6027bc5c5109e5...|259dac757896d24d7...|                    8775|     mogi das cruzes|            SP|        Sudeste|        Sudeste|
|4f2d8ab171c80ec83...|345ecd01c38d18a90..

###DEFININDO A NOVA TABELA CRIADA NA VARIAVEL DF_CLIENTES 

In [ ]:
df_clientes = spark.sql("""
SELECT  *,
        CASE
          WHEN customer_state IN ('MG', 'SP', 'RJ', 'ES') THEN 'Sudeste' 
          WHEN customer_state IN ('SC', 'RS', 'PR') THEN 'Sul' 
          WHEN customer_state IN ('DF', 'GO', 'MT', 'MS') THEN 'Centro-Oeste'
          WHEN customer_state IN ('AC', 'AM', 'AP', 'RO', 'RR', 'PA', 'TO') THEN 'Norte'
          ELSE 'Nordeste'
        END AS customer_region
FROM    clientes
"""

)

###EXCLUINDO A VIEW CLIENTES DEFINIDA ANTERIORMENTE

In [ ]:
spark.sql("DROP TABLE clientes")

DataFrame[]

###DEFININDO A NOVA VIEW CLIENTE

In [ ]:
df_clientes.createOrReplaceTempView("clientes")

###EXIBINDO A NOVA TABELA DF_CLIENTES

In [ ]:
df_clientes.show()

+--------------------+--------------------+------------------------+--------------------+--------------+---------------+
|         customer_id|  customer_unique_id|customer_zip_code_prefix|       customer_city|customer_state|customer_region|
+--------------------+--------------------+------------------------+--------------------+--------------+---------------+
|06b8999e2fba1a1fb...|861eff4711a542e4b...|                   14409|              franca|            SP|        Sudeste|
|18955e83d337fd6b2...|290c77bc529b7ac93...|                    9790|sao bernardo do c...|            SP|        Sudeste|
|4e7b3e00288586ebd...|060e732b5b29e8181...|                    1151|           sao paulo|            SP|        Sudeste|
|b2b6027bc5c5109e5...|259dac757896d24d7...|                    8775|     mogi das cruzes|            SP|        Sudeste|
|4f2d8ab171c80ec83...|345ecd01c38d18a90...|                   13056|            campinas|            SP|        Sudeste|
|879864dab9bc30475...|4c93744516

###JUNTANDO O DATAFRAME DF_CLIENTES E DF_ORDERS PELO ID USANDO O INNER JOIN

In [ ]:
df_orders_clientes = spark.sql("""
SELECT    *
FROM      orders
  INNER JOIN  clientes
    ON orders.customer_id = clientes.customer_id
""").show()

+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+--------------------+--------------------+------------------------+-------------------+--------------+---------------+
|            order_id|         customer_id|order_status|order_purchase_timestamp|  order_approved_at|order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|         customer_id|  customer_unique_id|customer_zip_code_prefix|      customer_city|customer_state|customer_region|
+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+--------------------+--------------------+------------------------+-------------------+--------------+---------------+
|a44895d095d7e0702...|000161a058600d590...|   delivered|     2017-07-16 0

###GRAVANDO ARQUIVO

In [ ]:
df_clientes.coalesce(1).write.option('header', True).csv("clientes_por_reg")

##IMPORTANDO O ARQUIVO GRAVADO PARA O DATALAKE

In [ ]:
from azure.storage.blob import BlobClient

connection_string = "DefaultEndpointsProtocol=https;AccountName=datalakeaula141600;AccountKey=YjT3wcnrtaF99X2Bzo1nMnBbJGGeNiK5ybrGYjZ/kKs+w9D+px/WO6a35FrLR5FyCLIwCL9qEbhiiTWy09L4RA==;EndpointSuffix=core.windows.net"

blob = BlobClient.from_connection_string(conn_str=connection_string, container_name="datalake", blob_name="/raw-zone/olist/olist_customers_dataset/olist_customers_dataset_with_region.csv")

with open("clientes_por_reg/part-00000-e13bc05a-7c86-4d34-b12f-1a30e897fda7-c000.csv", "rb") as data:
    blob.upload_blob(data)
